In [1]:
# Start writing code here...
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import igraph

This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [2]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
os.listdir("../data")

['sources.csv.gz',
 'input_wagon.xlsx',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'wagon_mode_compat.csv.gz',
 'description.txt',
 'output_wagon.xlsx',
 'metrics.csv.gz',
 'standplaces.csv.gz']

In [5]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz")
sources = pd.read_csv("../data/sources.csv.gz")
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz")

In [11]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.common import *

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ba67d69-ce72-437b-8bf4-6d3c9b906c4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>